# Physics Informed Neural Networks (PINNs) - 2D Wave Equation

Copyright (c) 2022 UKAEA. All rights reserved.

Copyright (c) 2022 Graphcore Ltd. All rights reserved.

This notebook contains a modified version of the work in https://github.com/farscape-project/PINNs_Benchmark/blob/main/PINN_Wave_TF-2.ipynb, modified by Graphcore Ltd so that it can be run with the latest version of Graphcore's [Poplar (TM) SDK](https://docs.graphcore.ai/projects/sdk-overview/).

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This file has been modified by Graphcore Ltd.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)


| Framework    |  Domain           | Tasks      | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|--------------|-------------------|------------|-------|----------|----------|--------------|--------------|
| TensorFlow 2 | AI for Simulation | PDE solver | MLP   | None     | training, inference | 4 | ~5 minutes  |

## Methods

This example shows training a Neural Network to converge towards a well-defined solution of a PDE by way of minimising for the residuals across the spatio-temporal domain.

Initial and Boundary conditions are met by introducing them into the loss function along with the PDE residuals. 

The PDE is defined over 2D spatial (`x`, `y`) and 1D temporal (`t`) dimensions, with the time dimension divided into a finite number of time-steps. 

Numerical Method - Spectral Solver using FFT. <br>
Code taken from http://people.bu.edu/andasari/courses/numericalpython/Week12Lecture21/Spectral_wave2.py

Equation: 

```
u_tt = u_xx + u_yy on [-1,1] x [-1,1] 
```

Dirichlet Boundary Conditions : 
```
u=0
```

Initial Distribution :
```
 u(x,y,t=0) = exp( -40((x-0.4)^2 + y^2) )
 ```

Initial Velocity Condition : 
```
u_t(x,y,t=0) = 0
```


## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs, since everything is already set up for you.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/MUD3GR) UPDATE ME WITH LINK!!!

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Install dependencies

First we need to install the required Python libraries.

In [ ]:
%pip install -r requirements.txt

## Numerical Solution

In this first step we build the numerical solution by solving the Wave Equation using a spectral solver implemented on the CPU using numpy.

The solution will not form the training data but will be used for comparing against the PINN solution.

First we import the necessary Python libraries:

In [ ]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
class WaveEquation:
    def __init__(self, N, T):
        self.N = N
        self.T = T
        self.x0 = -1.0
        self.xf = 1.0
        self.y0 = -1.0
        self.yf = 1.0
        self.initialization()
        self.initCond() 
        
    def initialization(self):
        k = np.arange(self.N + 1)
        self.x = np.cos(k*np.pi/self.N)
        self.y = self.x.copy()
        self.xx, self.yy = np.meshgrid(self.x, self.y)
        
        self.dt = 6/self.N**2
        self.plotgap = round((1/3)/self.dt)
        self.dt = (1/3)/self.plotgap
        
    def initCond(self):
        self.vv = np.exp(-40*((self.xx-0.4)**2 + self.yy**2))
        self.vvold = self.vv.copy()

    def solve(self):
        u_list = []  
        tc = 0
        nstep = round(self.T/self.dt) + 1 # nstep = round(3*self.plotgap+1)
        
        while tc < nstep:   
            xxx = np.arange(self.x0, self.xf+1/16, 1/16)
            yyy = np.arange(self.y0, self.yf+1/16, 1/16)
            vvv = interpolate.interp2d(self.x, self.y, self.vv, kind='cubic')
            Z = vvv(xxx, yyy)
            xxf, yyf = np.meshgrid(np.arange(self.x0,self.xf+1/16,1/16), np.arange(self.y0,self.yf+1/16,1/16))         
            uxx = np.zeros((self.N+1, self.N+1))
            uyy = np.zeros((self.N+1, self.N+1))
            ii = np.arange(1, self.N)
            
            for i in range(1, self.N):
                v = self.vv[i,:]
                V = np.hstack((v, np.flipud(v[ii])))
                U = np.fft.fft(V).real
                r1 = np.arange(self.N)
                r2 = 1j*np.hstack((r1, 0, -r1[:0:-1]))*U
                W1 = np.fft.ifft(r2).real
                s1 = np.arange(self.N+1)
                s2 = np.hstack((s1, -s1[self.N-1:0:-1]))
                s3 = -s2**2*U
                W2 = np.fft.ifft(s3).real
                uxx[i,ii] = W2[ii]/(1-self.x[ii]**2) - self.x[ii]*W1[ii]/(1-self.x[ii]**2)**(3/2)
                
            for j in range(1, self.N):
                v = self.vv[:,j]
                V = np.hstack((v, np.flipud(v[ii])))
                U = np.fft.fft(V).real
                r1 = np.arange(self.N)
                r2 = 1j*np.hstack((r1, 0, -r1[:0:-1]))*U
                W1 = np.fft.ifft(r2).real
                s1 = np.arange(self.N+1)
                s2 = np.hstack((s1, -s1[self.N-1:0:-1]))
                s3 = -s2**2*U
                W2 = np.fft.ifft(s3).real
                uyy[ii,j] = W2[ii]/(1-self.y[ii]**2) - self.y[ii]*W1[ii]/(1-self.y[ii]**2)**(3/2)
                
            vvnew = 2*self.vv - self.vvold + self.dt**2*(uxx+uyy)
            self.vvold = self.vv.copy()
            self.vv = vvnew.copy()
            tc += 1
            u_list.append(Z)
        return np.asarray(u_list) 

In [ ]:
def main(N, T):
    simulator = WaveEquation(N, T) 
    u_sol = simulator.solve()
    return u_sol

    
if __name__ == "__main__":
    time_end = 1 #Upto how many seconds we would like to simulate. 
    N = 30 #Spatial Discretisation. 
    dt =  6/N**2 #N and dt are fixed for ensuring numerical stability. 
    k = 30 + 1

    u_sol = main(N, time_end)    
    

    lb = np.asarray([-1.0, -1.0, 0]) #[x, y, t] Lower Bounds of the domain
    ub = np.asarray([1.0, 1.0, time_end]) #Upper Bounds of the domain/ 
    
    x = np.arange(-1, 1+1/16, 1/16)
    y = x.copy()
    t = np.arange(lb[2], ub[2]+dt, dt)
    
    xx, yy = np.meshgrid(x, y)
    u_ic = np.exp(-40*((xx-0.4)**2 + yy**2)) #Initial Gaussian Distribution. 
    
    U_sol = u_sol

    grid_length = len(x)
    
    #Storing the problem and solution information.
    df_dict = {'x': x,
               'y': y,
               't': t,
               'lower_range': lb,
               'upper_range': ub,
               'U_sol': U_sol}

# Training a PINN to Solve the 2D Wave Equation

Now we will train a PINN to provide a solution to the 2D wave equation defined above. There is no dataset, since the loss functions are based on well-defined initial and boundary conditions, and the PDE describing the system over its domain is known.

First, we import the necessary Python libraries:

In [ ]:
import os
import time
from tqdm.notebook import tqdm 
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import matplotlib.animation as animation
%matplotlib widget
plt.rcParams['figure.dpi'] = 250
import operator
from functools import reduce 

from pyDOE import lhs
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers, activations
from tensorflow.keras import backend as K 
from tensorflow.python import ipu

In [ ]:
# Setting the random seed. 
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## PINN Model

We define the PINN model using the Keras Model API.

The model consists of five fully-connected layers, with the four hidden layers using a `tanh` activation function.

In [ ]:
class PINN(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(100, activation=tf.nn.tanh)
    self.dense2 = tf.keras.layers.Dense(100, activation=tf.nn.tanh)
    self.dense3 = tf.keras.layers.Dense(100, activation=tf.nn.tanh)
    self.dense4 = tf.keras.layers.Dense(100, activation=tf.nn.tanh)
    self.output_layer = tf.keras.layers.Dense(1)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    x = self.output_layer(x)

    return x

## Latin Hypercube Sampling (LHS)

LHS sampling is used to generate random points over the `(x,y,t)` domain of interest for the PDE.

These coordinates, as well as similarly-generated points representing the initial conditions (`t=0`) and boundary conditions (`x=x_min`, `x=x_max`, `y=y_min`, `y=y_max`), will be used to evaluate the performance of the PINN during training.

In [ ]:
#Specifying the Domain of Interest. 
x_range = [-1.0, 1.0]
y_range = [-1.0, 1.0]
t_range = [0.0, time_end]
D = 1.0

lb = np.asarray([x_range[0], y_range[0], t_range[0]]) #lower bounds
ub = np.asarray([x_range[1], y_range[1], t_range[1]]) #Upper bounds

#Fucnction to sample collocation points across the spatio-temporal domain using a Latin Hypercube
def LHS_Sampling(N):
    return lb + (ub-lb)*lhs(3, N)

## Loss Functions: Evaluating the PDE and its initial/boundary conditions

We define multiple loss functions for the PINN model, each representing either the PDE or one of its initial/boundary conditions.

#### PDE Loss Function:

In [ ]:
#Domain Loss Function - measuring the deviation from the PDE functional. 
def pde(X):
    x = X[:, 0:1]
    y = X[:, 1:2]
    t = X[:, 2:3]
    u = model(tf.concat([x,y,t], 1))

    u_x = tf.gradients(u,x)[0]
    u_xx = tf.gradients(u_x, x)[0]
    u_y = tf.gradients(u,y)[0]
    u_yy = tf.gradients(u_y, y)[0]
    u_t = tf.gradients(u,t)[0]
    u_tt = tf.gradients(u_t, t)[0]
    pde_loss = u_tt - D*(u_xx + u_yy)

    return tf.reduce_mean(tf.square(pde_loss))

#### Boundary Condition Loss Function:

In [ ]:
#Boundary Loss Function - measuring the deviation from boundary conditions for f(x_lim, y_lim, t)
def boundary(X):
    u = model(X)
    bc_loss = u - 0
    
    return tf.reduce_mean(tf.square(bc_loss))

#### Initial Condition Loss Functions:

In [ ]:
#Initial Velocity Conditions :
def initial_velocity(X):
    x = X[:, 0:1]
    y = X[:, 1:2]
    t = X[:, 2:3]
    u = model(tf.concat([x,y,t], 1))

    u_t = tf.gradients(u, t)[0]
    initial_cond_loss = u_t

    return tf.reduce_mean(tf.square(initial_cond_loss))

#Reconstruction Loss Function - measuring the deviation from the actual output. Used to calculate the initial loss
def reconstruction(X, Y):
    u = model(X)
    recon_loss = u-Y
    return tf.reduce_mean(tf.square(recon_loss))

## Preparing the input data

Here we prepare the input tensors for the initial (`t=0`), boundary and domain regions.

In [ ]:
# Samples taken from each region for optimisation purposes. 
N_i = 1000 #Initial
N_b = 1000 #Boundary
N_f = 40000 #Domain
batch_size = 10000 # Split the Domain samples into batches

In [ ]:
# Preparing the inputs and outputs for comparison with the numerical solution. 
u = np.asarray(u_sol)
X, Y = np.meshgrid(x, y)
XY_star = np.hstack((X.flatten()[:,None], Y.flatten()[:,None]))
T_star = np.expand_dims(np.repeat(t, len(XY_star)), 1)
X_star_tiled = np.tile(XY_star, (len(t), 1))

X_star = np.hstack((X_star_tiled, T_star))
u_actual = np.expand_dims(u.flatten(),1)


In [ ]:
# Data for Initial Input
X_IC = np.hstack((XY_star, np.zeros(len(XY_star)).reshape(len(XY_star), 1)))
u_IC = u[0].flatten()
u_IC = np.expand_dims(u_IC, 1)

idx = np.random.choice(X_IC.shape[0], N_i, replace=False) 
X_i = X_IC[idx]
u_i = u_IC[idx]

In [ ]:
# Data for Boundary Input

X_left = LHS_Sampling(N_b)
X_left[:,0:1] = x_range[0]

X_right = LHS_Sampling(N_b)
X_right[:,0:1] = x_range[1]

X_bottom = LHS_Sampling(N_b)
X_bottom[:,1:2] = y_range[0]

X_top = LHS_Sampling(N_b)
X_top[:,1:2] = y_range[1]

X_b = np.vstack((X_right, X_top, X_left, X_bottom))
np.random.shuffle(X_b) 

In [ ]:
#Data for Domain Input
X_f = LHS_Sampling(N_f)

In [ ]:
#Converting to tensors 
X_i = tf.convert_to_tensor(X_i, dtype=tf.float32)
Y_i = tf.convert_to_tensor(u_i, dtype=tf.float32)
X_b = tf.convert_to_tensor(X_b, dtype=tf.float32)
X_f = tf.convert_to_tensor(X_f, dtype=tf.float32)

## TensorFlow Dataset

We wrap the input data inside a TensorFlow Dataset object, allowing us to easily batch the data as well as utilise optimisation techniques such as dataset prefetching.

In [ ]:
def create_datasets():
    # We split X_f up into batches in order to fit into IPU memory.
    # Each batch is fed to a different IPU in order to train in a data-parallel fashion
    num_batches = N_f // batch_size
    print("Batching the domain into {} batches of size {}".format(num_batches, batch_size))
    X_f_batched = tf.split(X_f, num_batches, axis=0)
    
    def gen():
        for i in range(num_batches):
            yield X_i, X_b, X_f_batched[i]
    
    # Wrap in TF dataset so Poplar knows what shapes to expect
    ds = tf.data.Dataset.from_generator(gen,
                                        output_shapes=((N_i, 3), (4*N_b, 3), (N_f // num_batches, 3)),
                                        output_types=(tf.float32, tf.float32, tf.float32))
    
    # Performance optimisations (using TF dataset allows us to use these)
    ds = ds.batch(1, drop_remainder=True)
    ds = ds.repeat().prefetch(100)
    
    # Inference dataset
    X_star_cast = tf.convert_to_tensor(X_star, dtype=tf.float32)
    ds_star = tf.data.Dataset.from_tensor_slices((X_star_cast)).batch(164439, drop_remainder=True).repeat()
    
    return ds, ds_star

In [ ]:
ds, ds_star = create_datasets()

## Model Training Function

The `train_step` function defines the part of our script which we wish to run on the IPU itself. As part of each training step step we perform a forward pass, calculate the losses and gradients, and perform the weight update over the model.

### On-Device Loop: `steps_per_execution`

On the IPU we can loop over `steps_per_execution` training steps in a single call to the hardware, by placing the code inside a `for` loop.

As a result, the IPU spends a greater fraction of the time doing compute and less time waiting for data from the host, thus improving performance.

In [ ]:
@tf.function(jit_compile=True)
def train_step(model, iterator, optimizer, outfeed_queue, steps_per_execution):
    for _ in tf.range(steps_per_execution):
        X = next(iterator)
        X_i = X[0][0]
        X_b = X[1][0]
        X_f = X[2][0]
        
        with tf.GradientTape() as tape:
            initial_loss = reconstruction(X_i, Y_i) + initial_velocity(X_i)
            boundary_loss = boundary(X_b)
            domain_loss = pde(X_f)

            loss = initial_loss + boundary_loss + domain_loss   

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        outfeed_queue.enqueue([initial_loss, boundary_loss, domain_loss])

## IPUConfig: Configure the IPU Device

We construct an instance of the `IPUConfig` class to request one or multiple IPUs and attach our program to these.

We request `4` IPUs, but the model can be trained using just a single IPU. As such, TensorFlow will automatically replicate the model across the IPUs, allowing for easy data-parallel training.

We also create an `IPUStrategy`, which allows us to control which parts of our program runs on the IPU.

In [ ]:
# configure IPU system
num_ipus = 4
cfg = ipu.config.IPUConfig()
cfg.auto_select_ipus = num_ipus
cfg.configure_ipu_system()

strategy = ipu.ipu_strategy.IPUStrategy()

We construct the model inside the scope of this IPU Strategy, indicating that the entire model be compiled for and placed on the IPU device.

In [ ]:
with strategy.scope():
    model = PINN()
    model.build(input_shape=(None,3))
    model.summary()

## Training Loop

Now we can train the model, looping over the total number of training steps in multiples of `steps_per_execution`.

We construct the optimiser and the `iterator` and `outfeed_queue` objects used to send data to/from the IPU asynchronously.

In [ ]:
epochs = 20000 #Number of Epochs for the training loop.
steps_per_execution = 2000 #Number of training steps to perform in an on-device loop
loss_list = []

start_time = time.time()

with strategy.scope():
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3, beta_1=.90)
    optimizer = ipu.optimizers.CrossReplicaOptimizer(optimizer)
    
    iterator = iter(ds)
    
    outfeed_queue = ipu.ipu_outfeed_queue.IPUOutfeedQueue()
    
    for it in range(0, epochs, steps_per_execution):
        strategy.run(train_step, args=[model, iterator, optimizer, outfeed_queue, steps_per_execution])

        initial_loss, boundary_loss, domain_loss = outfeed_queue.dequeue()
        loss_list.extend(initial_loss + boundary_loss + domain_loss)
        
        print('Epoch: %d/%d, Init: %.3e, Bound: %.3e, Domain: %.3e' \
              % (it+steps_per_execution, epochs, np.mean(initial_loss), np.mean(boundary_loss), np.mean(domain_loss)))


train_time = time.time() - start_time

## Plotting the Training Loss

The plot below shows how the overall training loss decreases as the model trains

In [ ]:
loss_plot = plt.figure(figsize=(5, 3))
plt.plot(loss_list)
plt.xlabel('Iterations')
plt.ylabel('L2 Error')

## Model Inference

The function below defines an inference step, performing only a forward pass over the model and returning the outputs.

In [ ]:
@tf.function(jit_compile=True)
def inference_step(model, iterator, outfeed_queue):
    X = next(iterator)
    u_pred = model(X)
    
    outfeed_queue.enqueue(u_pred)

The test dataset contains an odd number of elements, and we can process it in a single batch on the IPU. As such we use a single IPU, and so we create a fresh IPU Config.

In [ ]:
# New IPUConfig for inference
cfg = ipu.config.IPUConfig()
cfg.auto_select_ipus = 1
cfg.configure_ipu_system()

strategy = ipu.ipu_strategy.IPUStrategy()

## Model Inference

We then run inference over the test dataset and compute the L2 Error between the PINN output and the numerical solution calculated earlier.

In [ ]:
with strategy.scope():
    iterator = iter(ds_star)
    
    outfeed_queue = ipu.ipu_outfeed_queue.IPUOutfeedQueue()
    
    strategy.run(inference_step, args=[model, iterator, outfeed_queue])
    
    u_pred = outfeed_queue.dequeue()

l2_error = np.mean((u_actual - u_pred)**2)

print('Training Time: %d seconds, L2 Error: %.3e' % (train_time, l2_error))

u_pred = u_pred.numpy().reshape(len(u), grid_length, grid_length)

## Comparing the Results with the Numerical Solution

Finally we can visually compare the PINN-generated results with the numerical solution calculated previously.

In [ ]:
u_field = u_sol
fig = plt.figure(figsize=[5,3])
ax = fig.add_subplot(2,3,1)
ax.imshow(u_field[0])
ax.title.set_text('Initial')
ax.set_ylabel('Solution')
ax = fig.add_subplot(2,3,2)
ax.imshow(u_field[int(len(t)/2)])
ax.title.set_text('Middle')
ax = fig.add_subplot(2,3,3)
ax.imshow(u_field[-1])
ax.title.set_text('Final')

u_field = u_pred
ax = fig.add_subplot(2,3,4)
ax.imshow(u_field[0])
ax.set_ylabel('PINN')
ax = fig.add_subplot(2,3,5)
ax.imshow(u_field[int(len(t)/2)])
ax = fig.add_subplot(2,3,6)
ax.imshow(u_field[-1])

Cooler: Animation shows progression of numerical and PINN solutions over time.

In [ ]:
def plt_data():
    for i in range(0, len(t), 2):
        data_sol = u_sol[i]
        data_pred = u_pred[i]

        yield i, data_sol, data_pred

def update_plot(data):
    frame_idx, data_sol, data_pred = data

    im_sol.set_data(data_sol)
    im_sol.autoscale()
    im_pred.set_data(data_pred)
    im_pred.autoscale()
    plt.suptitle('Time Step {} of {}'.format(frame_idx, len(t)))
    return im_sol, im_pred

In [ ]:
fig = plt.figure(figsize=[5,3])

ax_sol = fig.add_subplot(1,2,1)
im_sol = ax_sol.imshow(u_sol[0])
ax_sol.title.set_text('Solution')

ax_pred = fig.add_subplot(1,2,2)
im_pred = ax_pred.imshow(u_pred[0])
ax_pred.title.set_text('PINN')
plt.suptitle('Time Step {} of {}'.format(0, len(t)))

anim = animation.FuncAnimation(fig, update_plot, plt_data, blit=True, interval=50,repeat=True)

Smoother: we can save this as a gif to display more smoothly

In [ ]:
writergif = animation.PillowWriter(fps=10)
anim.save('PINN.gif', dpi=600, writer=writergif)

In [ ]:
from IPython.display import Image
Image("PINN.gif")